# SCIBERT

trained on Zora

In [1]:
import spacy
import pandas as pd
from pathlib import Path

from spacy.cli.train import train as spacy_train

In [2]:
train_set = 'zora'
test_set = 'zora'
model_name = 'scibert'

base_path = Path.cwd().parent

# Edit paths as needed:
config_path = base_path / f"cfg/spacy/{model_name}.cfg"
model_out_path = base_path / f"data/models/ft-{model_name}-{train_set}"
train_path, dev_path, test_path = (
    base_path / f"data/spacy_docs/{train_set}_train.spacy",
    base_path / f"data/spacy_docs/{train_set}_dev.spacy",
    base_path / f"data/spacy_docs/{test_set}_test.spacy"
)

In [3]:
# train
spacy_train(
    config_path,
    output_path=model_out_path,
    use_gpu=0,
    overrides={
        "paths.train": str(train_path),
        "paths.dev": str(dev_path)
    }
)

ℹ Saving to output directory:
/home/dvdblk/dev/_thesis/experiments/data/models/ft-scibert-zora
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================


/home/dvdblk/dev/_thesis/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the che

✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'textcat']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  -------------  ------------  ----------  ------
  0       0           0.00          0.06        0.37    0.00
  6     200           0.00         37.34       21.37    0.21
 12     400           0.02         18.65       31.12    0.31
 18     600           0.02          4.41       31.34    0.31
 25     800           0.01          1.30       30.53    0.31
 31    1000           0.01          0.75       29.29    0.29
 37    1200           0.01          0.31       36.69    0.37
 43    1400           0.01          0.20       36.37    0.36
 50    1600           0.02          0.13       37.37    0.37
 56    1800           0.01          0.08       35.97    0.36
 62    2000           0.01          0.08       37.37    0.37
 68    2200           0.02          0.08

In [6]:
# test
best_model = base_path / f"data/models/ft-scibert-zora/model-best"
test_cats = [str(i) for i in range(1, 18)]
output_file= f"{best_model}/test_{test_set}_eval.json"

!python3 -m spacy benchmark accuracy --gpu-id 0 \
  {best_model}/ {test_path} \
  --output {output_file}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ℹ Using GPU: 0

================================== Results ==================================

TOK                 100.00
TEXTCAT (macro F)   50.18 
SPEED               5933  


=========================== Textcat F (per label) ===========================

          P        R        F
1      0.00     0.00     0.00
2     50.00    50.00    50.00
3     75.00    69.23    72.00
4    100.00   100.00   100.00
5    100.00   100.00   100.00
6      0.00     0.00     0.00
7    100.00   100.00   100.00
8     41.18    87.50    56.00
9     66.67    50.00    57.14
10    50.00    60.00    54.55
11     0.00     0.00     0.00
12    42.86    33.33    37.50
13   100.00    66.67    80.00
14     0.00     0.00     0.00
15    77.14   100.00    87.10
16    71.43    50.00    58.82
17     0.00     0.00     0.00


======================== Textcat ROC AUC (per label) ========================

     ROC AUC
1       0.90
2       0.99
3       0.97
4       1.00
5       1.00
6       0.83
7       1.00
8       0.96
9    

In [7]:

def get_test_df(file):
  test_df = pd.read_csv(file, sep='\t', keep_default_na=False,
                        index_col=0, encoding='utf-8')
  test_df = test_df.astype({'sdg': 'string',
                            'abstract': 'string'})
  test_df.drop(columns=['faculty', 'year'], inplace=True)

  return test_df

# Edit `test_df` path:
test_df = get_test_df(f"data/train_test/{test_set}_test_clean.tsv")

X_test = test_df['abstract'].values
y_test = test_df['sdg'].values

nlp = spacy.load(best_model)
print("Making predictions....")

spacy_probs = [doc.cats for doc in nlp.pipe(X_test)]
print("Done making predictions!")

# For each item, select the label to which the model has assigned the highest probability:
preds = []
probs = []
for label_probs_dict in spacy_probs:
    pred, prob = max(label_probs_dict.items(), key=lambda x: x[1])
    preds.append(pred)
    probs.append(prob)

preds = pd.Series(preds)
probs = pd.Series(probs)

FileNotFoundError: [Errno 2] No such file or directory: 'data/train_test/zora_test_clean.tsv'

In [11]:
preds_df = pd.DataFrame({'abstract': X_test,
                         'label': y_test,
                         'prediction': preds,
                         'probability': probs})
# Align original indices
preds_df.index = test_df.index

preds_df = preds_df.astype({'abstract': 'string',
                            'label': 'int',
                            'prediction': 'int',
                            'probability': 'float'})

preds_file = base_path / f"data/{train_set}-{test_set}_preds.tsv"
preds_df.to_csv(preds_file, sep='\t', encoding='utf-8')